In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import cv2
import math  # for mathematical operations
from keras.preprocessing import image   # for preprocessing the images
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from keras.applications.vgg19 import VGG19
from sklearn import metrics
import tensorflow as tf
import glob
from tqdm.notebook import tqdm
import os
from keras.layers import *
from keras import Model
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image

In [ ]:
from PIL import Image
from PIL import ImageFilter

In [ ]:
TF_WEIGHTS_PATH = '../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels.h5'
TF_WEIGHTS_PATH_NO_TOP = '../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
def VGG_19(include_top=True, weights='imagenet',input_tensor=None):
    input_shape = (None, None, 3)
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor)
        else:
            img_input = input_tensor
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(1000, activation='softmax', name='predictions')(x)

    model = Model(img_input, x)
    if include_top:
        weights_path = TF_WEIGHTS_PATH
    else:
        weights_path = TF_WEIGHTS_PATH_NO_TOP
    model.load_weights(weights_path)
    return model

In [ ]:
DATA_FOLDER = '../input/deepfake-detection-challenge'
TRAIN_SAMPLE_FOLDER = 'train_sample_videos'
TEST_FOLDER = 'test_videos'

print(f"Train samples: {len(os.listdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)))}")
print(f"Test samples: {len(os.listdir(os.path.join(DATA_FOLDER, TEST_FOLDER)))}")

In [ ]:
train_list = list(os.listdir(os.path.join(DATA_FOLDER, TRAIN_SAMPLE_FOLDER)))
ext_dict = []
for file in train_list:
    file_ext = file.split('.')[1]
    if (file_ext not in ext_dict):
        ext_dict.append(file_ext)
print(f"Extensions: {ext_dict}")      

In [ ]:
def get_img(images):
    global boxes,scores,num_detections
    im_heights,im_widths=[],[]
    imgs=[]
    for image in images:
        (im_height,im_width)=image.shape[:-1]
        imgs.append(image)
        im_heights.append(im_height)
        im_widths.append(im_widths)
    imgs=np.array(imgs)
    (boxes, scores_) = sess.run(
        [boxes_tensor, scores_tensor],
        feed_dict={image_tensor: imgs})
    finals=[]
    for x in range(boxes.shape[0]):
        scores=scores_[x]
        max_=np.where(scores==scores.max())[0][0]
        box=boxes[x][max_]
        ymin, xmin, ymax, xmax = box
        (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                      ymin * im_height, ymax * im_height)
        left, right, top, bottom = int(left), int(right), int(top), int(bottom)
        image=imgs[x]
        finals.append(cv2.cvtColor(cv2.resize(image[max([0,top-40]):bottom+80,max([0,left-40]):right+80],(240,240)),cv2.COLOR_BGR2RGB))
    return finals
def detect_video(video):
    frame_count=10
    capture = cv2.VideoCapture(video)
    v_len = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idxs = np.linspace(0,v_len,frame_count, endpoint=False, dtype=np.int)
    imgs=[]
    i=0
    for frame_idx in range(int(v_len)):
        ret = capture.grab()
        if not ret: 
            print("Error grabbing frame %d from movie %s" % (frame_idx, video))
        if frame_idx >= frame_idxs[i]:
            if frame_idx-frame_idxs[i]>20:
                return None
            ret, frame = capture.retrieve()
            if not ret or frame is None:
                print("Error retrieving frame %d from movie %s" % (frame_idx, video))
            else:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                imgs.append(frame)
            i += 1
            if i >= len(frame_idxs):
                break
    imgs=get_img(imgs)
    if len(imgs)<10:
        return None
    return np.hstack(imgs)
def get_birghtness(img):
    return img/img.max()
def process_img(img,flip=False):
    imgs=[]
    for x in range(10):
        if flip:
            imgs.append(get_birghtness(cv2.flip(img[:,x*240:(x+1)*240,:],1)))
        else:
            imgs.append(get_birghtness(img[:,x*240:(x+1)*240,:]))
    return np.array(imgs)

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile('../input/mobilenet-face/frozen_inference_graph_face.pb', 'rb')as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
cm = detection_graph.as_default()
cm.__enter__()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(graph=detection_graph, config=config)
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
boxes_tensor = detection_graph.get_tensor_by_name('detection_boxes:0')
scores_tensor = detection_graph.get_tensor_by_name('detection_scores:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

In [ ]:
os.mkdir('../train_videos_cut/')

In [ ]:
for x in tqdm(glob.glob('../input/deepfake-detection-challenge/train_sample_videos/*.mp4')):
    try:
        filename=x.replace('../input/deepfake-detection-challenge/train_sample_videos/','').replace('.mp4','.jpg')
        a=detect_video(x)
        if a is None:
            continue
        cv2.imwrite('../train_videos_cut/'+filename,a)
    except Exception as err:
        print(err)

In [ ]:
os.mkdir('../test_videos_cut/')

In [ ]:
for x in tqdm(glob.glob('../input/deepfake-detection-challenge/test_videos/*.mp4')):
    try:
        filename=x.replace('../input/deepfake-detection-challenge/test_videos/','').replace('.mp4','.jpg')
        a=detect_video(x)
        if a is None:
            continue
        cv2.imwrite('../test_videos_cut/'+filename,a)
    except Exception as err:
        print(err)

In [ ]:
train_imgs=glob.glob("../train_videos_cut/*.jpg")
train_imgs

In [ ]:
plt.imshow(plt.imread('../train_videos_cut/eqvuznuwsa.jpg', ))
plt.show() 

In [ ]:
test_imgs=glob.glob("../test_videos_cut/*.jpg")
test_imgs

In [ ]:
plt.imshow(plt.imread('../test_videos_cut/zmxeiipnqb.jpg', ))
plt.show() 

In [ ]:
json_file = [file for file in train_list if  file.endswith('json')][0]
print(f"JSON file: {json_file}")

In [ ]:
def get_meta_from_json(path):
    df = pd.read_json(os.path.join(DATA_FOLDER, path, json_file))
    df = df.T
    return df

meta_train_df = get_meta_from_json(TRAIN_SAMPLE_FOLDER)

In [ ]:
meta = np.array(list(meta_train_df.index))
storage = np.array([file for file in train_list if  file.endswith('mp4')])
print(f"Metadata: {meta.shape[0]}, Folder: {storage.shape[0]}")
print(f"Files in metadata and not in folder: {np.setdiff1d(meta,storage,assume_unique=False).shape[0]}")
print(f"Files in folder and not in metadata: {np.setdiff1d(storage,meta,assume_unique=False).shape[0]}")

In [ ]:
train_labels = []
for i in train_imgs:
    name = i.split('/')[2].replace('.jpg','.mp4')
    train_labels.append(meta_train_df[meta_train_df.index==name].iloc[0,0])

In [ ]:
train_data = pd.DataFrame()
train_data['class'] = train_labels
train_data['img_name']=train_imgs
train_data

In [ ]:
new_data = []
new_class = []
for file in tqdm(train_data.loc[train_data['class']=='REAL', 'img_name']):
    im=Image.open(file)
    im=im.convert("RGB")
    r,g,b=im.split()
    r=r.convert("RGB")
    g=g.convert("RGB")
    b=b.convert("RGB")
    im_blur=im.filter(ImageFilter.GaussianBlur)
    im_unsharp=im.filter(ImageFilter.UnsharpMask)
    r.save('..'+file.split('.')[-2]+'_r'+'.jpg')
    new_data.append('..'+file.split('.')[-2]+'_r'+'.jpg')
    new_class.append('REAL')
    g.save('..'+file.split('.')[-2]+'_g'+'.jpg')
    new_data.append('..'+file.split('.')[-2]+'_g'+'.jpg')
    new_class.append('REAL')
    b.save('..'+file.split('.')[-2]+'_b'+'.jpg')
    new_data.append('..'+file.split('.')[-2]+'_b'+'.jpg')
    new_class.append('REAL')
    im_blur.save('..'+file.split('.')[-2]+'_bl'+'.jpg')
    new_data.append('..'+file.split('.')[-2]+'_bl'+'.jpg')
    new_class.append('REAL')
    im_unsharp.save('..'+file.split('.')[-2]+'_un'+'.jpg')
    new_data.append('..'+file.split('.')[-2]+'_un'+'.jpg')
    new_class.append('REAL')
    
new_train_data = pd.DataFrame()
new_train_data['img_name'] = new_data
new_train_data['class'] = new_class
train_data=train_data.append(new_train_data, ignore_index=True)
train_data

In [ ]:
from collections import Counter
Counter(train_data["class"])

In [ ]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train_data.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img(train_data['img_name'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

In [ ]:
y = pd.get_dummies(train_data['class'])
base_model = VGG_19(weights='imagenet', include_top=False)
X = base_model.predict(X)
model = Sequential()
model.add(Dense(1024, activation = 'relu',  input_shape = (25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
X = X.reshape(785, 7*7*512)
max = X.max()
X = X/max

In [ ]:
model.fit(X, y, epochs=200, callbacks=[mcp_save], batch_size=128)

In [ ]:
test_image = []

# for loop to read and store frames
for i in tqdm(range(len(test_imgs))):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img(test_imgs[i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    test_image.append(img)
    
# converting the list to numpy array
X_test = np.array(test_image)

# shape of the array
X_test.shape

In [ ]:
base_model = VGG_19(weights='imagenet', include_top=False)
X_test = base_model.predict(X_test)

In [ ]:
X_test = X_test.reshape(400, 7*7*512)
max = X_test.max()
X_test = X_test/max

In [ ]:
result = model.predict(X_test)

In [ ]:
filename=[test_imgs[i].split('/')[-1].split('.')[0]+'.mp4' for i in range(len(test_imgs))]


In [ ]:
fake_prediction=[result[i][0] for i in range(len(result))]


In [ ]:
sample_submission = pd.read_csv("../input/deepfake-detection-challenge/sample_submission.csv")
test_files=glob.glob('./test_videos/*.jpg')
submission=pd.DataFrame()
submission['filename']=filename
submission['label']=fake_prediction


In [ ]:
submission.to_csv('submission.csv', index=False)